# Анализ тональности отзывов. Часть 1 из 2. Подготовка данных.

# Постановка задачи

В этом задании вам нужно воспользоваться опытом предыдущих недель, чтобы побить бейзлайн в соревновании по сентимент-анализу отзывов на товары на Kaggle Inclass:

https://inclass.kaggle.com/c/product-reviews-sentiment-analysis-light (старый лидерборд)

https://www.kaggle.com/c/simplesentiment

#### Review criteria:
В качестве ответа в этом задании вам нужно загрузить ноутбук с решением и скриншот вашего результата на leaderboard.

Убедитесь, что:

1. ход вашего решения задокументирован достаточно подробно для того, чтобы ваши сокурсники поняли, что вы делали и почему,

2. ваша команда в соревновании состоит только из вас и названа вашим логином на Сoursera, чтобы ваши сокурсники могли понять, что на скриншоте именно ваш результат

# Загрузка пакетов и данных

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
print(np.__version__)

import os
print(os.listdir("../input"))
np.random.seed(42)

In [ ]:
import nltk
import re
import seaborn as sns
import matplotlib.pyplot as plt
from random import randint
from time import sleep
from itertools import tee

Загрузим тренировочную и тестовую выбобрку в датафрейм, а затем объединим в один датафрейм

In [ ]:
df_train = pd.read_csv('../input/data-sentiment-analysis/products_sentiment_train.tsv', 
                       sep = '\t', header = None, names = ['text', 'label'])

In [ ]:
df_test = pd.read_csv('../input/data-sentiment-analysis/products_sentiment_test.tsv', sep = '\t')
df_test.columns = ['Id', 'text']

In [ ]:
df = pd.DataFrame({'text': pd.concat([df_train['text'], df_test['text']],axis = 0)})
df.shape

# EDA

Посмотрим на распределение классов и количество слов в текстах

In [ ]:
sns.countplot(df_train['label']);
plt.title('Train: Target distribution');

Есть небольшой дисбаланс классов: с признаком 1 почти в два раза больше. Можно будет попробовать приемы по исправлению дисбаланса, например, oversampling.

In [ ]:
fig = plt.figure(figsize = (15, 5));
ax1 = fig.add_subplot(121);
df_train['text'].apply(lambda x: len(x.split())).hist(bins = 20);
plt.title('Train: Number of words');

ax2 = fig.add_subplot(122);
df_test['text'].apply(lambda x: len(x.split())).hist(bins = 20);
plt.title('Test: Number of words');

В трейне и тесте распрделения более или менее похожи.

Посмотрим на другие характеристики обучающей и тестовой выборках

In [ ]:
print('Среднее кол-во слов в обучении и тесте:', 
      df_train['text'].apply(lambda x: len(x.split())).mean(), ' и ',
      df_test['text'].apply(lambda x: len(x.split())).mean())

In [ ]:
print('Медианы кол-ва слов в обучении и тесте:', 
      df_train['text'].apply(lambda x: len(x.split())).median(), ' и ',
      df_test['text'].apply(lambda x: len(x.split())).median())

In [ ]:
print('Мера разброса кол-ва слов в обучении и тесте:', 
      df_train['text'].apply(lambda x: len(x.split())).std(), ' и ',
      df_test['text'].apply(lambda x: len(x.split())).std())

In [ ]:
print('Всего слов в трейновой выборке:', df_train['text'].apply(lambda x: len(x.split())).sum())
print('Всего слов в тестовой выборке:', df_test['text'].apply(lambda x: len(x.split())).sum())

Взглянем на несколько случайных примеров.

In [ ]:
indices = np.random.randint(low = 0, high = len(df_train), size = 20)
df_train.iloc[indices]['text'].values

Все буквы приведены к нижнему регистру. <br>
Есть также небуквенные символы, которые будем обрабатывать в дальнейшем. <br>
Можно также попробовать заметить опечатки, поэтому качество текстов можно будет попробовать улучшить пакетом, который сопоставляет правильные слова ошибочным, например pyspellchecker или autocorrect.spell

In [ ]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

# Обогащение текстов
Обогатим обучающую выборку комментариями полученными с помощью аугментации через API Google translator. А именно первым действие сделаем перевод с английского на испанский (или русский), а вторым действием полученное предложение переведем обратно на оригинальный язык.

In [ ]:
%%capture
!pip install googletrans

In [ ]:
from googletrans import Translator
translator = Translator(service_urls=['translate.google.com',  'translate.google.es', 'translate.google.ru'])

У гугл переводчика есть особенности и он выдает ошибку на некоторые комбинации спецсимволов. Поэтому сделаем функцию по замене этих символов.

In [ ]:
def substitute_special_symb(mystring):
    return re.sub('&#', '', mystring)

Посмотрим на одном случайном примере как это работает:

In [ ]:
print('Оригинальный вариант:', 'all in all i am pleased with the router .')

translations_es = translator.translate(['all in all i am pleased with the router .'], dest='es')
translations_en = translator.translate([d.text for d in translations_es], dest='en')

for translation in translations_en:
    print('Аугментированный текст:', translation.text)

In [ ]:
print('Оригинальный вариант:', 'all in all i am pleased with the router .')

translations_ru = translator.translate(['all in all i am pleased with the router .'], dest='ru')
translations_en = translator.translate([d.text for d in translations_ru], dest='en')

for translation in translations_en:
    print('Аугментированный текст:', translation.text)

Сделаем запросы в гугл переводчик партиями (размером chunk_size) через искусственные паузы (sleep(randint(1, 2))), чтобы не перегружать их систему, а распределить нагрузку равномерно. <br>
Будем использовать для преобразования испанский язык, т.к. он представляется наиболее представленным в гугл переводчике

In [ ]:
# augmented_train = []
# chunk_size = 50

# for i in range(38, np.int(len(df_train)/chunk_size)):
#    translations_es = translator.translate(df_train['text'].apply(substitute_special_symb).tolist()[chunk_size*i:chunk_size*(i+1)], dest='es')
#    sleep(randint(1, 5))
#    translations_en = translator.translate([d.text for d in translations_es], dest='en')
#    sleep(randint(1, 5))

#    for translation in translations_en:
#        augmented_train.append(translation.text)
        
#    print(i+1, ' iteration has been ended')

Эта процедура была проделана лишь раз. Результаты были сохранены в файлы и далее в коде будет подгружаться файлы. Это быстрее и не надо лишний раз обращаться к гугл транслейту и нагружать их ресурсы.

Сохраним аугментированный текст для дальнейшего использования без вызова google translate и в дальнейшем будем использовать только этот файл, чтобы не обращаться к гугл транслейт еще раз.

In [ ]:
# with open('augmented_train.txt', 'w') as f:
#     for item in augmented_train:
#         f.write("%s\n" % item)

Сделаем тоже самое для тестовой выборки. Есть идея, что после преобразования гугл переводчиком опечатки будут исправлены и качество данных может быть улучшено и соответственно предсказания будут точнее.

In [ ]:
# augmented_test = []
# chunk_size = 50

# for i in range(8, np.int(len(df_test)/chunk_size)):
#     translations_es = translator.translate(df_test['text'].apply(substitute_special_symb).tolist()[chunk_size*i:chunk_size*(i+1)], dest='es')
#     sleep(randint(1, 4))
#     translations_en = translator.translate([d.text for d in translations_es], dest='en')
#     sleep(randint(1, 4))

#     for translation in translations_en:
#         augmented_test.append(translation.text)
        
#     print(i+1, ' iteration has been ended')

In [ ]:
# with open('augmented_test.txt', 'w') as f:
#     for item in augmented_test:
#         f.write("%s\n" % item)

Загружаем ранее аугментированный текст двумя порциями - обучающей и тестовой

In [ ]:
print(os.listdir("../input/augmented-data"))

In [ ]:
df_train_augmented = pd.read_csv('../input/augmented-data/augmented_train.txt', sep = '\n', 
                              header = None, names = ['text'])
df_train_augmented['label'] = df_train.label

In [ ]:
df_test_augmented = pd.read_csv('../input/augmented-data/augmented_test.txt', sep = '\n', 
                              header = None, names = ['text'])

Добавляем аугментированный текст в датафрейм

In [ ]:
df['text_translated'] = pd.DataFrame({'text': pd.concat([df_train_augmented['text'], df_test_augmented['text']], axis = 0)})
df.shape

### Обогатим текст с помощью предобученных веторов word2vec
Для каждого слова из нашего текста будем добавлять ближайшие синонимы (соседи) согласно косинусной меры между их векторным представлением

In [ ]:
%%capture
!pip install autocorrect

In [ ]:
%%capture
!pip install pyspellchecker

In [ ]:
from gensim.models import Word2Vec, KeyedVectors
from autocorrect import spell, Speller
from spellchecker import SpellChecker

Загрузим word2vec вектора, обученные на твитах и гугл новостях

In [ ]:
%%time
# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
# path = '../input/gensim-word-vectors/'
GLOVE_TWITTER = '../input/gensim-word-vectors/glove-twitter-100/glove-twitter-100'
twitter_model = KeyedVectors.load_word2vec_format(GLOVE_TWITTER)

In [ ]:
%%time
# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
GOOGLE_NEWS = '../input/googles-trained-word2vec-model-in-python/GoogleNews-vectors-negative300.bin.gz'
news_model = KeyedVectors.load_word2vec_format(GOOGLE_NEWS, binary=True)

Напишем функцию, которая получает на вход фразу и для каждого слова из фразы подбирает 5 ближайших соседей в векторном пространстве по косинусной мере. Если слово из входящей фразы не найдено в векторном пространстве, то пытаемся найти ошибку в написании и исправить пакетом autocorrect и далее по исправленному слову ищем соседей в векторном пространстве:

In [ ]:
def get_similar_bag_of_words_ordered(phrase, topn = 5, model = twitter_model):  
    bag_words = []
    ordered_bag_words = []
    # считаем вектор по каждому слову из фразы
    for word in phrase.split():
        try:
            bag_words.append([item[0] for item in model.most_similar([word], topn = topn)]) 
        except KeyError:
            try:
                subbag = [item[0]  for item in model.most_similar([spell(word)], topn = topn-1)]
                subbag.append(spell(word))
                bag_words.append(subbag)
            except KeyError:
                continue
                
    for i in range(topn):
        for w in bag_words:
            ordered_bag_words.append(w[i])
        
    return ' '.join([w for w in ordered_bag_words])

In [ ]:
get_similar_bag_of_words_ordered('treee grow slowly')

## Функция для обработки текстов
Preprocessor - приводит к нижнему регистру, убирает знаки препинания и т.д.

In [ ]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

Применим наши функцию к текстам

с моделью на основе твитов

In [ ]:
%%time
%%capture

# на основе твитов
df['text_w2v_twitter'] = df['text'].apply(preprocessor).apply(get_similar_bag_of_words_ordered)

с моделью на основе новостей

In [ ]:
%%time
%%capture

# на основе новостей
def get_similar_bag_of_words_news_ordered(phrase):
    return get_similar_bag_of_words_ordered(phrase = phrase, topn = 5, model = news_model)

df['text_w2v_news'] = df['text'].apply(preprocessor).apply(get_similar_bag_of_words_news_ordered)

Проверяем содержание полученного датафрейма

In [ ]:
print(df.shape)

In [ ]:
df.head()

Сохраняем полученный датафрейм в файл

In [ ]:
df.to_csv('enriched_train_test_text.csv', index = 'false')

### Идеи, которые не успел попробовать

1. Расширить выборку train. Например, датасетом из imdb (from keras.datasets import imdb). 

2. Для обогащения выборки использовать модель word2vec обученную на биграмах - https://www.kaggle.com/s4sarath/word2vec-unigram-bigrams-